## Generate images for literature review section of the thesis paper to help illustrate SDF and implicit density fields on a relevant design from the SimJEB dataset

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
google_drive_path = ''
os.chdir(google_drive_path)
print("Current working directory:", os.getcwd())
!ls


!python -m pip install trimesh
!python -m pip install pyvista
!python -m pip install rtree



Mounted at /content/gdrive
Current working directory: /content/gdrive/Othercomputers/My Mac/Code
 Dataset_generation		    Optimizers
 figures			    requirements.txt
 figures_cs			    results_distributed2d
 File_Paths			    results_distributed2d_0.7
 Functions			    results_distributed2d_keep_edges_soft
'Function Validation and testing'   results_distributed2d_mdke
 hard_GINN_results		    SimJEB_Data
 JEB_topo_0.3			    Test_Cases
 JEB_topo_0.6			    Trained_Models
 Litterature_Review		    Trained_Models_final
 Loss_functions			    Training_Functions
 Models				    Training_Log
 Model_Training_Scripts
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 11.7 MB/s eta 0:00:00


In [ ]:

step_file = "Code/SimJEB_Data/SimJEB Step_Files/411.stp"
surf_mesh = "Code/SimJEB_Data/SimJEB_surfmesh_(obj)/411.obj"

out_dir = "./figures"       
PIXELS = (2400, 1800)       
EDGE_TUBE_REL = 0.002       
MESH_EDGE_WIDTH = 0.9       
VOXELS_ACROSS = 96          
N_POINTCLOUD = 100_000      
OFF_SCREEN = True           


import os
os.makedirs(out_dir, exist_ok=True)

import numpy as np
import pyvista as pv
import trimesh as tm

from OCP.STEPControl import STEPControl_Reader
from OCP.IFSelect import IFSelect_RetDone
from OCP.BRepMesh import BRepMesh_IncrementalMesh
from OCP.TopExp import TopExp_Explorer
from OCP.TopAbs import TopAbs_FACE, TopAbs_EDGE
from OCP.BRep import BRep_Tool
from OCP.TopLoc import TopLoc_Location
from OCP.gp import gp_Pnt
from OCP.Bnd import Bnd_Box
from OCP.BRepBndLib import BRepBndLib
from OCP.TopoDS import TopoDS
from OCP.BRepAdaptor import BRepAdaptor_Curve  

pv.global_theme.background = "white"
pv.global_theme.show_edges = False
pv.global_theme.font.family = "arial"
pv.global_theme.smooth_shading = True
pv.global_theme.multi_samples = 8

try:
    pv.start_xvfb()
except Exception:
    pass

# ----------------- helpers -----------------
def read_step_shape(path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(path)
    if status != IFSelect_RetDone:
        raise RuntimeError(f"Failed to read STEP: {path}")
    reader.TransferRoots()
    return reader.OneShape()

def shape_bounds(shape):
    box = Bnd_Box()
    BRepBndLib.Add_s(shape, box, True)
    xmn, ymn, zmn, xmx, ymx, zmx = box.Get()
    return (xmn, xmx, ymn, ymx, zmn, zmx)

def tessellate_shape(shape, rel_deflection=1/220, angle=0.35):
    xmin, xmax, ymin, ymax, zmin, zmax = shape_bounds(shape)
    max_dim = max(xmax - xmin, ymax - ymin, zmax - zmin)
    deflection = max_dim * rel_deflection
    mesher = BRepMesh_IncrementalMesh(shape, deflection, True, angle, True)
    mesher.Perform()

def shape_surface_to_pv(shape):
    points_accum, faces_accum = [], []
    point_offset = 0
    face_exp = TopExp_Explorer(shape, TopAbs_FACE)
    while face_exp.More():
        f_shp = face_exp.Current(); face_exp.Next()
        loc = TopLoc_Location()
        tri = None
        try:
            tri = BRep_Tool.Triangulation_s(TopoDS.Face_s(f_shp), loc)
        except AttributeError:
            tri = BRep_Tool.Triangulation(TopoDS.Face_s(f_shp), loc)
        if tri is None:
            continue
        n_nodes = tri.NbNodes()
        T = loc.Transformation()
        face_points = []
        for i in range(1, n_nodes + 1):
            p = tri.Node(i).Transformed(T) if hasattr(tri, "Node") else tri.Nodes().Value(i).Transformed(T)
            face_points.append([p.X(), p.Y(), p.Z()])
        n_tris = tri.NbTriangles()
        for i in range(1, n_tris + 1):
            t = tri.Triangle(i) if hasattr(tri, "Triangle") else tri.Triangles().Value(i)
            i1, i2, i3 = t.Get()
            faces_accum.extend([3, point_offset + i1 - 1,
                                   point_offset + i2 - 1,
                                   point_offset + i3 - 1])
        points_accum.extend(face_points)
        point_offset += n_nodes
    if not points_accum:
        raise RuntimeError("No triangulation on STEP shape.")
    surface = pv.PolyData(np.asarray(points_accum, float),
                          np.asarray(faces_accum, np.int64))
    return surface

def edges_to_polylines(shape, n_samples=120):
    """Sample *all* edges in 3D using BRepAdaptor_Curve (works even for p-curves)."""
    lines = []
    exp = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp.More():
        e = TopoDS.Edge_s(exp.Current()); exp.Next()
        adaptor = BRepAdaptor_Curve(e)
        t0, t1 = adaptor.FirstParameter(), adaptor.LastParameter()
        ts = np.linspace(t0, t1, int(n_samples))
        pts = []
        for t in ts:
            p = adaptor.Value(t)  # gp_Pnt in 3D
            pts.append([p.X(), p.Y(), p.Z()])
        if len(pts) < 2:
            continue
        poly = pv.PolyData()
        arr = np.asarray(pts, float)
        poly.points = arr
        poly.lines = np.hstack(([arr.shape[0]], np.arange(arr.shape[0], dtype=np.int64))).astype(np.int64)
        lines.append(poly)
    return lines

def compute_camera_from_bounds(bounds, margin=1.6):
    xmin, xmax, ymin, ymax, zmin, zmax = bounds
    cx, cy, cz = (0.5*(xmin+xmax), 0.5*(ymin+ymax), 0.5*(zmin+zmax))
    size = np.array([xmax-xmin, ymax-ymin, zmax-zmin])
    r = float(size.max()) * margin
    return dict(position=(cx + r, cy + r, cz + r),
                focal_point=(cx, cy, cz),
                viewup=(0.0, 0.0, 1.0))

def set_plotter_camera(pl, bounds):
    cam = compute_camera_from_bounds(bounds, margin=1.6)
    pl.camera.position = cam["position"]
    pl.camera.focal_point = cam["focal_point"]
    pl.camera.up = cam["viewup"]
    pl.camera.parallel_projection = True
    pl.camera.zoom(1.08)

def new_plotter(title):
    pl = pv.Plotter(off_screen=OFF_SCREEN, window_size=PIXELS)
    pl.set_background("white")
    pl.add_text(title, font_size=16, color="black", position="upper_left")
    return pl

# ----------------- pipeline -----------------
shape = read_step_shape(step_file)
tessellate_shape(shape)
brep_surface = shape_surface_to_pv(shape)
brep_bounds = brep_surface.bounds
brep_edge_lines = edges_to_polylines(shape, n_samples=140)

mesh_pd = pv.read(surf_mesh)
mesh_tm = tm.load_mesh(surf_mesh, process=True)
if not isinstance(mesh_tm, tm.Trimesh):
    raise RuntimeError("Loaded OBJ is not a single Trimesh.")
mesh_bounds = mesh_pd.bounds

pl = new_plotter("B-Rep (STEP) with surface patch boundaries")
pl.add_mesh(brep_surface, color="#D9D9D9", smooth_shading=True, specular=0.1)

dx = brep_bounds[1]-brep_bounds[0]
dy = brep_bounds[3]-brep_bounds[2]
dz = brep_bounds[5]-brep_bounds[4]
diag = float(np.sqrt(dx*dx + dy*dy + dz*dz))
tube_radius = max(1e-9, EDGE_TUBE_REL * diag)  
for pline in brep_edge_lines:
    tube = pline.tube(radius=tube_radius, n_sides=16, capping=True)
    pl.add_mesh(tube, color="black", smooth_shading=False, specular=0.0)

set_plotter_camera(pl, brep_bounds)
pl.show(auto_close=False)
brep_path = os.path.join(out_dir, "brep.png")
pl.screenshot(brep_path); pl.close()
print(f"Saved: {brep_path}")

pl = new_plotter("Surface Mesh (triangles visible)")
pl.add_mesh(mesh_pd, color="#6AAFE6", smooth_shading=True,
            show_edges=True, edge_color="black",
            line_width=MESH_EDGE_WIDTH, specular=0.05)
set_plotter_camera(pl, mesh_bounds)
pl.show(auto_close=False)
mesh_path = os.path.join(out_dir, "mesh.png")
pl.screenshot(mesh_path); pl.close()
print(f"Saved: {mesh_path}")

x0, x1, y0, y1, z0, z1 = mesh_bounds
max_dim = max(x1 - x0, y1 - y0, z1 - z0)
pitch = float(max_dim / VOXELS_ACROSS)

vox = mesh_tm.voxelized(pitch=pitch)
vox_occ = vox.matrix.astype(np.uint8)

try:
    spacing = np.array(vox.pitch, dtype=float).ravel()
    spacing = np.repeat(spacing.item(), 3) if spacing.size == 1 else spacing
except Exception:
    spacing = np.array([pitch, pitch, pitch], dtype=float)

if hasattr(vox, "transform"):
    o_center = (vox.transform @ np.array([0.0, 0.0, 0.0, 1.0]))[:3]
else:
    o_center = np.array([x0, y0, z0], dtype=float)
origin_corner = o_center - 0.5 * spacing

grid = getattr(pv, "UniformGrid", getattr(pv, "ImageData"))()
grid.dimensions = np.array(vox_occ.shape, dtype=int) + 1
grid.origin = origin_corner
grid.spacing = tuple(spacing.tolist())
grid.cell_data["occupancy"] = vox_occ.flatten(order="F")
vox_surface = grid.threshold(0.5, scalars="occupancy")

pl = new_plotter(f"Voxel Representation (pitch = {pitch:.3g})")
pl.add_mesh(vox_surface, color="#B0BEC5", smooth_shading=False)
set_plotter_camera(pl, mesh_bounds)
pl.show(auto_close=False)
vox_path = os.path.join(out_dir, "voxels.png")
pl.screenshot(vox_path); pl.close()
print(f"Saved: {vox_path}")

# --- Render: Point cloud ---
np.random.seed(0)
pts = mesh_tm.sample(N_POINTCLOUD)
pc = pv.PolyData(pts)
pl = new_plotter(f"Point Cloud (N = {len(pts):,})")
pl.add_mesh(pc, render_points_as_spheres=True, point_size=2.0,
            color="black", smooth_shading=False)
set_plotter_camera(pl, mesh_bounds)
pl.show(auto_close=False)
pc_path = os.path.join(out_dir, "pointcloud.png")
pl.screenshot(pc_path); pl.close()
print(f"Saved: {pc_path}")

ModuleNotFoundError: No module named 'OCP'

In [ ]:
# ====================== COLAB SETUP ======================
!pip -q install pyvista trimesh pillow matplotlib

import os, numpy as np
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# --- path ---
step_file = "Code/SimJEB_Data/SimJEB Step_Files/411.stp"
surf_mesh = "Code/SimJEB_Data/SimJEB_surfmesh_(obj)/411.obj"

# Output directory on Drive
out_dir = "/content/gdrive/MyDrive/figures_cs"
os.makedirs(out_dir, exist_ok=True)
print("OBJ:", surf_mesh)
print("Out:", out_dir)

# GENERATE FIGURES 
import pyvista as pv
import trimesh as tm
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image

pv.start_xvfb()  
mpl.rcParams.update({
    "figure.facecolor": "white",
    "axes.facecolor": "white",
    "savefig.facecolor": "white",
    "axes.grid": False,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.spines.left": False,
    "axes.spines.bottom": False,
    "axes.titlesize": 16,
})

# Load geometry
mesh_pd = pv.read(surf_mesh)
mesh_tm = tm.load_mesh(surf_mesh, process=True)
assert isinstance(mesh_tm, tm.Trimesh), "OBJ must be a single Trimesh."

# ---------------- Parameters ----------------
RES = 512                   
MARGIN = 0.08               
DENSITY_ALPHA_FRAC = 0.02   
PIXELS_3D = (1200, 900)     

# Planes:
PLANE1_NORMAL = np.array([1.0, 0, 0], dtype=float)
PLANE2_NORMAL = np.array([0, 1.0, 0], dtype=float)
PLANE2_POINT  = np.array([-21.3572765, -88.9787065, 44.724602], dtype=float)

WORLD_UP = np.array([0, 0, 1.0], dtype=float)  

# ---------------- Helpers ----------------
def fixed_basis_for_normal(n):
    """Deterministic, upright basis (v = +Z when possible)."""
    n = n / np.linalg.norm(n)
    axis = int(np.argmax(np.abs(n)))
    if axis == 0:   
        u = np.array([0, 1, 0], float)  
        v = np.array([0, 0, 1], float)  
    elif axis == 1: 
        u = np.array([1, 0, 0], float)  
        v = np.array([0, 0, 1], float)  
    else:          
        u = np.array([1, 0, 0], float)  
        v = np.array([0, 1, 0], float)  
    n = np.sign(n[axis]) * np.eye(3)[axis]
    return u, v, n

def plane_name_from_normal(n):
    n = np.round(n/np.linalg.norm(n)).astype(int)
    if np.all(n == [1,0,0]) or np.all(n == [-1,0,0]):  return "YZ"
    if np.all(n == [0,1,0]) or np.all(n == [0,-1,0]):  return "XZ"
    if np.all(n == [0,0,1]) or np.all(n == [0,0,-1]):  return "XY"
    return "plane"

def plane_frame_extents(verts, center, u, v, margin):
    rel = verts - center
    xu, xv = rel @ u, rel @ v
    ux_min, ux_max = float(xu.min()), float(xu.max())
    vx_min, vx_max = float(xv.min()), float(xv.max())
    du, dv = ux_max-ux_min, vx_max-vx_min
    return (ux_min - margin*du, ux_max + margin*du), (vx_min - margin*dv, vx_max + margin*dv)

def sample_plane_grid(center, u, v, range_u, range_v, res):
    us = np.linspace(range_u[0], range_u[1], res)
    vs = np.linspace(range_v[0], range_v[1], res)
    UU, VV = np.meshgrid(us, vs, indexing='xy')
    pts = center + UU[...,None]*u + VV[...,None]*v
    return pts.reshape(-1,3), UU, VV

def signed_distance_mesh(mesh, points):
    try:
        from trimesh.proximity import ProximityQuery
        d = ProximityQuery(mesh).signed_distance(points)
        if np.isnan(d).any(): raise RuntimeError
        return d
    except Exception:
        from trimesh.proximity import nearest
        du = nearest.distance(mesh, points)
        inside = mesh.contains(points)
        du[inside] *= -1.0
        return du

def implicit_density_from_sdf(sdf, alpha):
    return 1.0 / (1.0 + np.exp(sdf / alpha))

def save_field_plot(field2d, extent, title, cmap, iso_value, out_path, vmin=None, vmax=None):
    fig, ax = plt.subplots(figsize=(6.8, 3.6), dpi=220)
    im = ax.imshow(field2d, origin='lower', extent=extent, cmap=cmap,
                   vmin=vmin, vmax=vmax, interpolation='bilinear', aspect='auto')
    ax.contour(field2d, levels=[iso_value], colors='k', linewidths=1.6,
               origin='lower', extent=extent)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_title(title)
    fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.08, fraction=0.18)
    plt.tight_layout()
    fig.savefig(out_path, bbox_inches='tight'); plt.close(fig)
    print("Saved:", out_path)

def render_geometry_with_plane(mesh_pd, center, normal, range_u, range_v, out_path, title):
    plane = pv.Plane(center=center, direction=normal,
                     i_size=(range_u[1]-range_u[0]),
                     j_size=(range_v[1]-range_v[0]))
    pl = pv.Plotter(off_screen=True, window_size=PIXELS_3D)
    pl.set_background("white")
    pl.add_text(title, font_size=16, color="black", position="upper_left")
    pl.add_mesh(mesh_pd, color="#CD7F32", smooth_shading=True, specular=0.2)
    pl.add_mesh(plane, color="#9AA3A7", opacity=0.55, smooth_shading=False)

    b = mesh_pd.bounds
    cx, cy, cz = (0.5*(b[0]+b[1]), 0.5*(b[2]+b[3]), 0.5*(b[4]+b[5]))
    diag = np.linalg.norm([b[1]-b[0], b[3]-b[2], b[5]-b[4]])
    pl.camera.position = (center + 1.8*diag*(normal/np.linalg.norm(normal) + np.array([0.6,0.4,0.3]))).tolist()
    pl.camera.focal_point = [cx, cy, cz]
    pl.camera.up = WORLD_UP.tolist()
    pl.camera.parallel_projection = True
    pl.camera.zoom(1.15)
    pl.show(auto_close=False); pl.screenshot(out_path); pl.close()
    print("Saved:", out_path)

def render_overlay_sdf(mesh_pd, mesh_tm, center, normal, u, v, range_u, range_v,
                       res, out_path, title):
    us = np.linspace(range_u[0], range_u[1], res)
    vs = np.linspace(range_v[0], range_v[1], res)
    P = center[None,None,:] + us[None,:,None]*u[None,None,:] + vs[:,None,None]*v[None,None,:]
    pts = P.reshape(-1,3)
    sdf = signed_distance_mesh(mesh_tm, pts).reshape(res, res)

    grid = pv.StructuredGrid()
    grid.points = P.reshape(-1,3, order='F')
    grid.dimensions = (res, res, 1)
    grid["sdf"] = sdf.T.flatten(order='F')

    m = float(np.nanmax(np.abs(sdf)))
    clim = (-m, +m)

    pl = pv.Plotter(off_screen=True, window_size=(1400, 1050))
    pl.set_background("white")
    pl.add_text(title, font_size=18, color="black", position="upper_left")
    pl.add_mesh(mesh_pd, color="#CFE6FF", opacity=0.95, smooth_shading=True, specular=0.15)
    pl.add_mesh(grid, scalars="sdf", cmap="jet", clim=clim, opacity=0.85, show_scalar_bar=False)
    pl.add_mesh(grid.contour(isosurfaces=[0.0], scalars="sdf"), color="white", line_width=3.0)

    # ===== Camera strictly NORMAL to the plane (orthographic) =====
    b = mesh_pd.bounds
    diag = np.linalg.norm([b[1]-b[0], b[3]-b[2], b[5]-b[4]])
    n_hat = normal / np.linalg.norm(normal)
    pl.camera.position = (center + 2.0*diag*n_hat).tolist()
    pl.camera.focal_point = center.tolist()
    # Choose an 'up' that's not parallel to normal
    pl.camera.up = (WORLD_UP if abs(np.dot(n_hat, WORLD_UP)) < 0.95 else u).tolist()
    pl.camera.parallel_projection = True
    pl.camera.zoom(1.10)

    pl.show(auto_close=False); pl.screenshot(out_path); pl.close()
    print("Saved:", out_path)

# ---------------- Compute & save ----------------
bounds = mesh_pd.bounds
center0 = np.array([(bounds[0]+bounds[1])*0.5,
                    (bounds[2]+bounds[3])*0.5,
                    (bounds[4]+bounds[5])*0.5])
max_dim = max(bounds[1]-bounds[0], bounds[3]-bounds[2], bounds[5]-bounds[4])
alpha = DENSITY_ALPHA_FRAC * max_dim
verts = np.asarray(mesh_tm.vertices)

panel_paths = []

# -------- Plane 1 (YZ, normal +X) --------
n1 = PLANE1_NORMAL
c1 = center0.copy()
u1, v1, n1 = fixed_basis_for_normal(n1)
ru1, rv1 = plane_frame_extents(verts, c1, u1, v1, MARGIN)
pts1, *_ = sample_plane_grid(c1, u1, v1, ru1, rv1, RES)
sdf1 = signed_distance_mesh(mesh_tm, pts1).reshape(RES, RES)
dens1 = implicit_density_from_sdf(sdf1, alpha=alpha)
extent1 = (ru1[0], ru1[1], rv1[0], rv1[1])

img = os.path.join(out_dir, "plane_1_YZ.png")
render_geometry_with_plane(mesh_pd, c1, n1, ru1, rv1, img, "Cross section plane (YZ)"); panel_paths.append(img)
m = float(np.nanmax(np.abs(sdf1)))
img = os.path.join(out_dir, "sdf_1_YZ.png")
save_field_plot(sdf1, extent1, "Signed Distance Field (iso=0)", 'jet', 0.0, img, vmin=-m, vmax=m); panel_paths.append(img)
img = os.path.join(out_dir, "density_1_YZ.png")
save_field_plot(dens1, extent1, "Implicit Density Field (iso=0.5)", 'jet', 0.5, img, vmin=0, vmax=1); panel_paths.append(img)

# -------- Plane 2 (XZ, normal +Y)  --------
n2 = PLANE2_NORMAL
c2 = PLANE2_POINT.copy() 
u2, v2, n2 = fixed_basis_for_normal(n2)
ru2, rv2 = plane_frame_extents(verts, c2, u2, v2, MARGIN)
pts2, *_ = sample_plane_grid(c2, u2, v2, ru2, rv2, RES)
sdf2 = signed_distance_mesh(mesh_tm, pts2).reshape(RES, RES)
dens2 = implicit_density_from_sdf(sdf2, alpha=alpha)
extent2 = (ru2[0], ru2[1], rv2[0], rv2[1])

img = os.path.join(out_dir, "plane_2_XZ.png")
render_geometry_with_plane(mesh_pd, c2, n2, ru2, rv2, img, "Cross section plane (XZ)"); panel_paths.append(img)
m = float(np.nanmax(np.abs(sdf2)))
img = os.path.join(out_dir, "sdf_2_XZ.png")
save_field_plot(sdf2, extent2, "Signed Distance Field (iso=0)", 'jet', 0.0, img, vmin=-m, vmax=m); panel_paths.append(img)
img = os.path.join(out_dir, "density_2_XZ.png")
save_field_plot(dens2, extent2, "Implicit Density Field (iso=0.5)", 'jet', 0.5, img, vmin=0, vmax=1); panel_paths.append(img)

# -------- 2×3 Plate --------
cols, rows, pad = 3, 2, 30
imgs = [Image.open(p).convert("RGB") for p in panel_paths]
W = max(im.size[0] for im in imgs); H = max(im.size[1] for im in imgs)
plate = Image.new("RGB", (cols*W + (cols+1)*pad, rows*H + (rows+1)*pad), (255,255,255))
for i, im in enumerate(imgs):
    r, c = divmod(i, cols)
    x = pad + c*(W + pad); y = pad + r*(H + pad)
    if im.size != (W, H): im = im.resize((W, H), Image.LANCZOS)
    plate.paste(im, (x, y))
plate_path = os.path.join(out_dir, "cross_sections_plate_2x3.png")
plate.save(plate_path)
print("\nSaved plate:", plate_path)

# -------- Extra: 3D overlay on Plane 2 
overlay_path = os.path.join(out_dir, "overlay_sdf_plane2.png")
render_overlay_sdf(mesh_pd, mesh_tm, c2, n2, u2, v2, ru2, rv2,
                   res=RES, out_path=overlay_path,
                   title="3D Geometry + SDF on Plane 2 (normal view, iso=0)")

Mounted at /content/gdrive
OBJ: /content/gdrive/Othercomputers/My Mac/Code/SimJEB_Data/SimJEB_surfmesh_(obj)/411.obj
Out: /content/gdrive/MyDrive/figures_cs
Mounted at /content/gdrive
OBJ: /content/gdrive/Othercomputers/My Mac/Code/SimJEB_Data/SimJEB_surfmesh_(obj)/411.obj
Out: /content/gdrive/MyDrive/figures_cs


/usr/local/lib/python3.12/dist-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Saved: /content/gdrive/MyDrive/figures_cs/plane_1_YZ.png
Saved: /content/gdrive/MyDrive/figures_cs/sdf_1_YZ.png
Saved: /content/gdrive/MyDrive/figures_cs/density_1_YZ.png
Saved: /content/gdrive/MyDrive/figures_cs/plane_2_XZ.png
Saved: /content/gdrive/MyDrive/figures_cs/sdf_2_XZ.png
Saved: /content/gdrive/MyDrive/figures_cs/density_2_XZ.png

Saved plate: /content/gdrive/MyDrive/figures_cs/cross_sections_plate_2x3.png
Saved: /content/gdrive/MyDrive/figures_cs/overlay_sdf_plane2.png
